In [10]:
import numpy as np
import pandas as pd
import os

In [11]:
def read_analyzed_edge(full_path):
    temp_df = pd.read_csv(full_path)
    temp_df['period_score'] = (1 < temp_df['period']) | (temp_df['period'] < 4)
    temp_df['maxcov_score'] = temp_df['maxcov'] < .2
    temp_df['amp_score'] = (4 < temp_df['amp']) |(temp_df['amp'] < 20)
    temp_df['sum_score'] = temp_df.loc[:,['period_score','maxcov_score','amp_score']].astype(int).sum(axis=1)
    temp_df['total_score'] = temp_df['sum_score'].sum()
    return temp_df

In [12]:
DATA_DIR = 'analyzed_edges/'
files = [file for file in os.listdir(DATA_DIR) if file.startswith('edge_analysis') and file.endswith('.csv')]
df_dict = dict()
for file in files[:10]:
    full_path = os.path.join(DATA_DIR, file)
    date = file.replace('.csv','')
    temp_df = read_analyzed_edge(full_path)
    display(temp_df.head())
    df_dict[date] = temp_df

,amp,freq,phase,offset,maxcov,period,std_dev,nonzeros,amp_err,freq_err,period_score,maxcov_score,amp_score,sum_score,total_score
0,-95207.979270,0.000252,1.554514,95203.311709,2.639480e+16,415.899194,40.413132,43.0,1.624648e+08,0.214865,True,False,True,2,96
1,-67.505780,0.010860,0.874646,63.224100,2.689060e+03,9.642533,43.068691,50.0,4.992712e+01,0.005606,True,False,True,2,96
2,-106.147829,0.008664,1.053691,103.003644,1.632427e+04,12.086728,45.982391,60.0,1.258295e+02,0.006420,True,False,True,2,96
3,-95.914066,0.009465,1.060498,93.247441,6.070130e+03,11.064345,48.364666,70.0,7.616453e+01,0.005100,True,False,True,2,96
4,-50.731682,0.015710,0.739674,46.953087,5.581782e+01,6.665727,49.490242,78.0,6.221161e+00,0.002522,True,False,True,2,96


,amp,freq,phase,offset,maxcov,period,std_dev,nonzeros,amp_err,freq_err,period_score,maxcov_score,amp_score,sum_score,total_score
0,-62.751339,0.017666,0.410184,50.066963,30.220159,5.927736,53.548492,74.0,4.452733,0.001552,True,False,True,2,96
1,-59.560361,0.020010,0.335291,45.981611,8.925562,5.233267,54.291447,84.0,2.854732,0.001147,True,False,True,2,96
2,-59.165524,0.021829,0.306308,45.317009,7.606364,4.797184,54.522377,94.0,2.757964,0.000925,True,False,True,2,96
3,-59.827218,0.021820,0.441964,48.128198,7.487102,4.799263,55.134708,104.0,2.736257,0.000890,True,False,True,2,96
4,-59.837744,0.022583,0.515913,49.985260,7.761386,4.637092,54.824692,114.0,2.785926,0.000835,True,False,True,2,96


,amp,freq,phase,offset,maxcov,period,std_dev,nonzeros,amp_err,freq_err,period_score,maxcov_score,amp_score,sum_score,total_score
0,-140268.959881,0.000278,1.549847,140258.411411,1.348474e+16,376.989648,48.212452,53.0,1.161238e+08,0.114999,True,False,True,2,96
1,-116.240052,0.010836,0.800138,105.105876,2.545865e+03,9.663847,50.849899,63.0,4.879980e+01,0.003024,True,False,True,2,96
2,-68.808378,0.016502,0.452294,55.591561,4.899282e+01,6.345756,52.312042,73.0,5.811201e+00,0.001528,True,False,True,2,96
3,-62.258477,0.019503,0.322430,47.525947,9.147317e+00,5.369517,53.197895,83.0,2.674612e+00,0.001045,True,False,True,2,96
4,-61.515885,0.021459,0.278853,46.120904,5.700542e+00,4.880085,53.645378,93.0,2.387581e+00,0.000811,True,False,True,2,96


,amp,freq,phase,offset,maxcov,period,std_dev,nonzeros,amp_err,freq_err,period_score,maxcov_score,amp_score,sum_score,total_score
0,-963.082909,0.002792,1.374639,955.720184,3.009961e+08,37.508041,46.515945,35.0,17346.884910,0.025581,True,False,True,2,96
1,-107.031473,0.009854,0.858431,99.138162,1.176464e+04,10.627574,50.828846,44.0,106.112266,0.006111,True,False,True,2,96
2,-56.344039,0.015732,0.501247,47.020103,1.629210e+02,6.656629,53.355591,50.0,10.873033,0.003089,True,False,True,2,96
3,-52.154471,0.017854,0.434998,42.329439,4.077179e+01,5.865312,54.850838,59.0,5.186783,0.002288,True,False,True,2,96
4,-53.428349,0.018186,0.507907,44.490569,2.735708e+01,5.758282,56.913074,68.0,4.384809,0.002061,True,False,True,2,96


,amp,freq,phase,offset,maxcov,period,std_dev,nonzeros,amp_err,freq_err,period_score,maxcov_score,amp_score,sum_score,total_score
0,66.877790,0.015924,-0.588377,25.940669,843.061126,6.576397,46.336174,170.0,27.106614,0.004605,True,False,True,2,96
1,37.830994,0.033568,-2.007087,66.172617,10.875597,3.119662,44.919914,180.0,3.297817,0.001519,True,False,True,2,96
2,35.079598,0.032042,-1.635789,66.178118,10.716168,3.268154,43.373947,190.0,3.273556,0.001554,True,False,True,2,96
3,32.462629,0.030511,-1.241908,66.111896,10.722361,3.432198,41.666248,200.0,3.274502,0.001513,True,False,True,2,96
4,29.571987,0.027752,-0.744741,65.489034,10.364705,3.773347,40.052429,210.0,3.219426,0.001615,True,False,True,2,96


,amp,freq,phase,offset,maxcov,period,std_dev,nonzeros,amp_err,freq_err,period_score,maxcov_score,amp_score,sum_score,total_score
0,-28.749176,0.012548,0.917834,27.214300,506.989754,8.345220,40.055283,28.0,20.828401,0.008290,True,False,True,2,96
1,-22.993294,0.015778,0.782020,21.408140,53.332382,6.637105,41.296337,32.0,6.087513,0.005802,True,False,True,2,96
2,-60.646478,0.008253,1.188659,61.918593,21467.272785,12.688991,44.247963,40.0,144.805456,0.012834,True,False,True,2,96
3,-87.592380,0.006993,1.269534,89.850800,86973.835498,14.975694,46.686764,47.0,293.345061,0.014295,True,False,True,2,96
4,-41.177763,0.011671,1.070004,43.781100,921.627133,8.972759,48.287754,53.0,28.834365,0.007492,True,False,True,2,96


,amp,freq,phase,offset,maxcov,period,std_dev,nonzeros,amp_err,freq_err,period_score,maxcov_score,amp_score,sum_score,total_score
0,-189043.579522,0.000210,1.553938,189034.865340,2.506229e+16,499.524952,44.276806,32.0,1.583107e+08,0.087786,True,False,True,2,96
1,-83264.046344,0.000308,1.548161,83257.434854,1.511943e+16,340.012727,46.624027,39.0,1.229611e+08,0.232941,True,False,True,2,96
2,-81.942783,0.011272,0.770797,74.953772,3.493385e+03,9.290430,50.102474,46.0,5.682301e+01,0.005328,True,False,True,2,96
3,-94.411775,0.010769,0.869316,87.853101,3.940121e+03,9.724381,53.132225,56.0,6.054645e+01,0.004758,True,False,True,2,96
4,-58.529258,0.015909,0.583649,50.670388,1.020566e+02,6.582417,54.961460,64.0,8.430607e+00,0.002613,True,False,True,2,96


,amp,freq,phase,offset,maxcov,period,std_dev,nonzeros,amp_err,freq_err,period_score,maxcov_score,amp_score,sum_score,total_score
0,335863.754130,0.000162,-1.584511,335853.847814,4.108672e+16,647.756733,42.124802,31.0,2.026986e+08,0.048785,True,False,True,2,96
1,-191409.220440,0.000209,1.555235,191400.339240,7.150041e+16,501.938855,45.074345,37.0,2.673956e+08,0.145739,True,False,True,2,96
2,-67629.979761,0.000364,1.544354,67622.600089,3.388211e+15,287.321782,48.870068,46.0,5.820834e+07,0.156891,True,False,True,2,96
3,-89.838580,-0.011395,2.348665,81.696891,2.347619e+03,-9.189673,51.026100,55.0,4.639098e+01,0.004130,True,False,True,2,96
4,-52.492631,0.018063,0.391200,41.986614,3.451793e+01,5.797490,52.686100,61.0,4.775014e+00,0.002082,True,False,True,2,96


,amp,freq,phase,offset,maxcov,period,std_dev,nonzeros,amp_err,freq_err,period_score,maxcov_score,amp_score,sum_score,total_score
0,-63.466929,0.019262,0.415447,50.997389,10.112520,5.436540,56.038826,88.0,2.865378,0.001125,True,False,True,2,96
1,-63.109209,0.020584,0.427353,51.008318,6.829229,5.087315,56.682804,98.0,2.613279,0.000934,True,False,True,2,96
2,-63.219584,0.021514,0.481995,52.276902,6.792856,4.867513,56.767799,108.0,2.606311,0.000828,True,False,True,2,96
3,-63.134561,0.021888,0.597404,54.301993,6.801106,4.784376,56.474136,118.0,2.607893,0.000794,True,False,True,2,96
4,-62.448580,0.021761,0.766799,56.550935,6.867080,4.812271,55.932362,128.0,2.620511,0.000832,True,False,True,2,96


,amp,freq,phase,offset,maxcov,period,std_dev,nonzeros,amp_err,freq_err,period_score,maxcov_score,amp_score,sum_score,total_score
0,-91.609660,0.012814,0.710149,81.125549,683.953763,8.172159,55.596199,66.0,24.323583,0.002676,True,False,True,2,92
1,-65.920065,0.017567,0.451034,53.440988,31.419174,5.961176,56.541939,76.0,4.541417,0.001536,True,False,True,2,92
2,-61.256831,0.020556,0.332294,47.281123,8.849914,5.094303,57.009968,85.0,2.974880,0.001112,True,False,True,2,92
3,-61.160018,0.022282,0.318109,46.945058,8.769312,4.699702,56.891418,95.0,2.961302,0.000908,True,False,True,2,92
4,-61.576745,0.023249,0.375459,48.536172,8.930296,4.504302,56.535732,105.0,2.988360,0.000817,True,False,True,2,92


In [26]:
def preprocess_yolo_dir(parent_dir, img_width=719, csv_path='date_labels.csv'):
    full_csv_path = os.path.join(parent_dir, csv_path)
    if os.path.exists(full_csv_path):
        print(f'File {full_csv_path} already found, returning')
        truth_df = pd.read_csv(full_csv_path)
        return truth_df
    
    truth_df_cols = list()
    for file in os.listdir(parent_dir):
        date = file.split('_', 1)[0].replace('.txt','')
        new_name = os.path.join(parent_dir, date + '.txt')
        print(new_name)
        if not os.path.exists(new_name): os.rename(os.path.join(parent_dir, file), new_name)
        with open(new_name, 'r') as f:
            truth_arr = np.zeros(img_width, dtype=np.bool_)
            for line in f.readlines():
                if not line: continue
                print(line)
                arr = np.array([float(x) for x in line.split(' ')])
                assert arr[0] == 0
                arr = arr[::2] # grab only the x values
                xmin = int(np.amin(arr) * img_width)
                xmax = int(np.amax(arr) * img_width)
                truth_arr[xmin:xmax] = True
            truth_df_cols.append(pd.DataFrame(truth_arr, columns=date))
    truth_df = pd.concatenate(truth_df_cols, axis=1)
    truth_df.to_csv(full_csv_path)
    print(f'File {full_csv_path} created, returning')
    return truth_df
    


File sample_yolo_labels/date_labels.csv already found, returning


In [ ]:
def window_label(arr, window_size=240, offset=10):
    labels = list()
    for i in range(0, len(arr) - window_size, offset):
        if np.count_nonzero(arr[i,i+window_size]) <= 0:
            labels.append(True)
        else:
            labels.append(False)
    return np.array(labels)

def concat_full_truth_values(truth_dir='sample_yolo_labels/', data_dir='analyzed_edges/', target_col='lstid_truth'):
    truth_df = preprocess_yolo_dir(truth_dir)
    files = [file for file in os.listdir(data_dir) if file.startswith('edge_analysis') and file.endswith('.csv')]
    df_list = list()
    for file in files[:10]:
        full_path = os.path.join(data_dir, file)
        date = file.replace('.csv','')
        temp_df = read_analyzed_edge(full_path)
        temp_df['date'] = date
        temp_df[target_col] = window_label(truth_df[date])
        temp_df.set_index('date', inplace=True)
        temp_df.index = pd.to_datetime(temp_df.index)
        df_list.append(temp_df)
    Xy = pd.concat(df_list, axis=0)
    return Xy.loc[:,Xy.columns != target_col], Xy.loc[:,Xy.columns == target_col]

X, y = concat_full_truth_values()
print(X.shape)
print(y.shape)

In [ ]:
from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier, RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score, accuracy_score, f1_score, average_precision, roc_auc
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.2)
model = LGBMClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('ROC', roc_auc(y_pred, y_test))

